<a href="https://colab.research.google.com/github/HarshitManik/Glucos_sparameter_frequencey_prediction/blob/main/Glucose_DLL_model_Trial_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import pandas as pd

# Upload the file
uploaded = files.upload()

# Read the CSV file into a DataFrame
for filename in uploaded.keys():
    df = pd.read_csv(filename)
    print(f"Uploaded file: {filename}")
    print(df.head())  # Display the first few rows of the dataframe


Saving BGL_DATASET_2.csv to BGL_DATASET_2.csv
Uploaded file: BGL_DATASET_2.csv
    FREQ        S11  bgl
0  0.000 -18.901456  400
1  0.015 -18.859446  400
2  0.030 -18.735411  400
3  0.045 -18.535068  400
4  0.060 -18.267139  400


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Load the data
data = pd.read_csv('BGL_DATASET_2.csv')

# Extract features (S11 and frequency) and target (bgl)
X = data[['FREQ', 'S11']].values
y = data['bgl'].values

# Normalize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reshape the data for LSTM input (samples, timesteps, features)
X = X.reshape((X.shape[0], 1, X.shape[1]))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the improved LSTM model
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='linear'))  # Output layer for regression

# Compile the model with Adam optimizer & learning rate scheduling
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

# Implement callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-5)

# Train the model
history = model.fit(X_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_data=(X_test, y_test),
                    verbose=2,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, mae = model.evaluate(X_test, y_test, verbose=0)
print(f'Test Loss: {loss}, Test MAE: {mae}')

# Predict on test data
y_pred = model.predict(X_test)

# Compare predictions with actual values
for i in range(10):
    print(f'Predicted: {y_pred[i][0]}, Actual: {y_test[i]}')


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
376/376 - 9s - 25ms/step - loss: 35955.5234 - mae: 158.4323 - val_loss: 14452.5635 - val_mae: 100.5851 - learning_rate: 0.0010
Epoch 2/100
376/376 - 3s - 7ms/step - loss: 13179.0957 - mae: 96.5158 - val_loss: 11136.7959 - val_mae: 91.4483 - learning_rate: 0.0010
Epoch 3/100
376/376 - 5s - 13ms/step - loss: 12525.5938 - mae: 94.5883 - val_loss: 10755.3037 - val_mae: 90.0743 - learning_rate: 0.0010
Epoch 4/100
376/376 - 3s - 7ms/step - loss: 12185.7051 - mae: 93.0079 - val_loss: 10569.0039 - val_mae: 89.2403 - learning_rate: 0.0010
Epoch 5/100
376/376 - 3s - 7ms/step - loss: 11900.2480 - mae: 91.9663 - val_loss: 10357.0225 - val_mae: 88.1492 - learning_rate: 0.0010
Epoch 6/100
376/376 - 3s - 8ms/step - loss: 11966.2041 - mae: 92.3774 - val_loss: 10113.8340 - val_mae: 86.9221 - learning_rate: 0.0010
Epoch 7/100
376/376 - 5s - 12ms/step - loss: 11453.6348 - mae: 90.2394 - val_loss: 9857.9854 - val_mae: 85.6972 - learning_rate: 0.0010
Epoch 8/100
376/376 - 3s - 7ms/step - loss: 

In [4]:
import numpy as np

# Actual values
y_actual = np.array([370, 130, 170, 250, 400, 200, 130, 170, 110, 270])

# Predicted values
y_pred = np.array([240.0679, 114.6367, 229.2858, 283.1296, 293.0371,
                   267.3064, 90.3147, 246.7037, 98.4353, 294.2297])

# Compute MAPE
mape = np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100

# Compute accuracy
accuracy = 100 - mape

print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 74.94%
